## 1. Load Spark Environment & Data

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.6/spark-3.5.6-bin-hadoop3.tgz
!tar -xvf spark-3.5.6-bin-hadoop3.tgz
!pip install -q findspark

spark-3.5.6-bin-hadoop3/
spark-3.5.6-bin-hadoop3/examples/
spark-3.5.6-bin-hadoop3/examples/jars/
spark-3.5.6-bin-hadoop3/examples/jars/spark-examples_2.12-3.5.6.jar
spark-3.5.6-bin-hadoop3/examples/jars/scopt_2.12-3.7.1.jar
spark-3.5.6-bin-hadoop3/examples/src/
spark-3.5.6-bin-hadoop3/examples/src/main/
spark-3.5.6-bin-hadoop3/examples/src/main/java/
spark-3.5.6-bin-hadoop3/examples/src/main/java/org/
spark-3.5.6-bin-hadoop3/examples/src/main/java/org/apache/
spark-3.5.6-bin-hadoop3/examples/src/main/java/org/apache/spark/
spark-3.5.6-bin-hadoop3/examples/src/main/java/org/apache/spark/examples/
spark-3.5.6-bin-hadoop3/examples/src/main/java/org/apache/spark/examples/mllib/
spark-3.5.6-bin-hadoop3/examples/src/main/java/org/apache/spark/examples/mllib/JavaDecisionTreeClassificationExample.java
spark-3.5.6-bin-hadoop3/examples/src/main/java/org/apache/spark/examples/mllib/JavaAssociationRulesExample.java
spark-3.5.6-bin-hadoop3/examples/src/main/java/org/apache/spark/examples/mllib/Jav

In [ ]:
import os

os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-3.5.6-bin-hadoop3'

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import mean, col, split, regexp_extract, when, lit
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import QuantileDiscretizer

In [ ]:
spark = SparkSession.builder.master('local[*]').appName('Saprk ML example on titanic data').getOrCreate()

In [ ]:
!wget https://github.com/rajeevratan84/datascienceforbusiness/raw/master/titanic.csv

--2025-09-13 13:20:05--  https://github.com/rajeevratan84/datascienceforbusiness/raw/master/titanic.csv
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rajeevratan84/datascienceforbusiness/master/titanic.csv [following]
--2025-09-13 13:20:06--  https://raw.githubusercontent.com/rajeevratan84/datascienceforbusiness/master/titanic.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85153 (83K) [text/plain]
Saving to: ‘titanic.csv’

titanic.csv         100%[===================>]  83.16K  --.-KB/s    in 0.02s   

2025-09-13 13:20:06 (3.71 MB/s) - ‘titanic.csv’ saved [85153/85153]



In [ ]:
titanic_df = spark.read.csv('titanic.csv', header='True', inferSchema='True')
titanic_df.head()

Row(pclass=1, name='Allen, Miss. Elisabeth Walton', sex='female', age=29.0, sibsp=0, parch=0, ticket='24160', fare=211.3375, cabin='B5', embarked='S', survived=1)

In [ ]:
titanic_df.show(5)

+------+--------------------+------+------+-----+-----+------+--------+-------+--------+--------+
|pclass|                name|   sex|   age|sibsp|parch|ticket|    fare|  cabin|embarked|survived|
+------+--------------------+------+------+-----+-----+------+--------+-------+--------+--------+
|     1|Allen, Miss. Elis...|female|  29.0|    0|    0| 24160|211.3375|     B5|       S|       1|
|     1|Allison, Master. ...|  male|0.9167|    1|    2|113781|  151.55|C22 C26|       S|       1|
|     1|Allison, Miss. He...|female|   2.0|    1|    2|113781|  151.55|C22 C26|       S|       0|
|     1|Allison, Mr. Huds...|  male|  30.0|    1|    2|113781|  151.55|C22 C26|       S|       0|
|     1|Allison, Mrs. Hud...|female|  25.0|    1|    2|113781|  151.55|C22 C26|       S|       0|
+------+--------------------+------+------+-----+-----+------+--------+-------+--------+--------+
only showing top 5 rows



In [ ]:
titanic_df.count()

1309

In [ ]:
titanic_df.printSchema()

root
 |-- pclass: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: double (nullable = true)
 |-- sibsp: integer (nullable = true)
 |-- parch: integer (nullable = true)
 |-- ticket: string (nullable = true)
 |-- fare: double (nullable = true)
 |-- cabin: string (nullable = true)
 |-- embarked: string (nullable = true)
 |-- survived: integer (nullable = true)



In [ ]:
titanic_df.select('pclass', 'embarked', 'survived').show()

+------+--------+--------+
|pclass|embarked|survived|
+------+--------+--------+
|     1|       S|       1|
|     1|       S|       1|
|     1|       S|       0|
|     1|       S|       0|
|     1|       S|       0|
|     1|       S|       1|
|     1|       S|       1|
|     1|       S|       0|
|     1|       S|       1|
|     1|       C|       0|
|     1|       C|       0|
|     1|       C|       1|
|     1|       C|       1|
|     1|       S|       1|
|     1|       S|       1|
|     1|       S|       0|
|     1|       C|       0|
|     1|       C|       1|
|     1|       C|       1|
|     1|       C|       0|
+------+--------+--------+
only showing top 20 rows



## Exploratory Data Analysis

In [ ]:
titanic_df.groupBy('Survived').count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  500|
|       0|  809|
+--------+-----+



In [ ]:
titanic_df.groupBy('Sex', 'Survived').count().show()

+------+--------+-----+
|   Sex|Survived|count|
+------+--------+-----+
|  male|       0|  682|
|female|       1|  339|
|female|       0|  127|
|  male|       1|  161|
+------+--------+-----+



In [ ]:
titanic_df.groupBy('pclass', 'Survived').count().show()

+------+--------+-----+
|pclass|Survived|count|
+------+--------+-----+
|     1|       0|  123|
|     3|       1|  181|
|     1|       1|  200|
|     2|       1|  119|
|     2|       0|  158|
|     3|       0|  528|
+------+--------+-----+



In [ ]:
def null_value_count(df):
  null_columns_counts = []
  numRows = df.count()
  for k in df.columns:
    nullRows = df.where(col(k).isNull()).count()
    if(nullRows > 0):
      temp = k,nullRows
      null_columns_counts.append(temp)
  return(null_columns_counts)

In [ ]:
null_columns_count_list = null_value_count(titanic_df)
null_columns_count_list

[('age', 263), ('fare', 1), ('cabin', 1014), ('embarked', 2)]

In [ ]:
spark.createDataFrame(null_columns_count_list, ['Column_With_Null_Value', 'Null_Values_Count']).show()

+----------------------+-----------------+
|Column_With_Null_Value|Null_Values_Count|
+----------------------+-----------------+
|                   age|              263|
|                  fare|                1|
|                 cabin|             1014|
|              embarked|                2|
+----------------------+-----------------+



In [ ]:
mean_age = titanic_df.select(mean('Age')).collect()[0][0]
print(mean_age)

29.8811345124283


In [ ]:
titanic_df.show()

+------+--------------------+------+------+-----+-----+--------+--------+-------+--------+--------+
|pclass|                name|   sex|   age|sibsp|parch|  ticket|    fare|  cabin|embarked|survived|
+------+--------------------+------+------+-----+-----+--------+--------+-------+--------+--------+
|     1|Allen, Miss. Elis...|female|  29.0|    0|    0|   24160|211.3375|     B5|       S|       1|
|     1|Allison, Master. ...|  male|0.9167|    1|    2|  113781|  151.55|C22 C26|       S|       1|
|     1|Allison, Miss. He...|female|   2.0|    1|    2|  113781|  151.55|C22 C26|       S|       0|
|     1|Allison, Mr. Huds...|  male|  30.0|    1|    2|  113781|  151.55|C22 C26|       S|       0|
|     1|Allison, Mrs. Hud...|female|  25.0|    1|    2|  113781|  151.55|C22 C26|       S|       0|
|     1| Anderson, Mr. Harry|  male|  48.0|    0|    0|   19952|   26.55|    E12|       S|       1|
|     1|Andrews, Miss. Ko...|female|  63.0|    1|    0|   13502| 77.9583|     D7|       S|       1|


In [ ]:
titanic_df = titanic_df.withColumn('Initial',regexp_extract(col('Name'),'([A-Za-z]+)\.',1))

<>:1: SyntaxWarning: invalid escape sequence '\.'
<>:1: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipython-input-3007629235.py:1: SyntaxWarning: invalid escape sequence '\.'
  titanic_df = titanic_df.withColumn('Initial',regexp_extract(col('Name'),'([A-Za-z]+)\.',1))


In [ ]:
titanic_df.show()

+------+--------------------+------+------+-----+-----+--------+--------+-------+--------+--------+-------+
|pclass|                name|   sex|   age|sibsp|parch|  ticket|    fare|  cabin|embarked|survived|Initial|
+------+--------------------+------+------+-----+-----+--------+--------+-------+--------+--------+-------+
|     1|Allen, Miss. Elis...|female|  29.0|    0|    0|   24160|211.3375|     B5|       S|       1|   Miss|
|     1|Allison, Master. ...|  male|0.9167|    1|    2|  113781|  151.55|C22 C26|       S|       1| Master|
|     1|Allison, Miss. He...|female|   2.0|    1|    2|  113781|  151.55|C22 C26|       S|       0|   Miss|
|     1|Allison, Mr. Huds...|  male|  30.0|    1|    2|  113781|  151.55|C22 C26|       S|       0|     Mr|
|     1|Allison, Mrs. Hud...|female|  25.0|    1|    2|  113781|  151.55|C22 C26|       S|       0|    Mrs|
|     1| Anderson, Mr. Harry|  male|  48.0|    0|    0|   19952|   26.55|    E12|       S|       1|     Mr|
|     1|Andrews, Miss. Ko...

In [ ]:
titanic_df.select('Initial').distinct().show()

+--------+
| Initial|
+--------+
|    Dona|
|     Don|
|    Miss|
|Countess|
|     Col|
|    Lady|
|     Rev|
|  Master|
|     Mme|
|    Capt|
|      Mr|
|      Dr|
|     Mrs|
|     Sir|
|Jonkheer|
|    Mlle|
|   Major|
|      Ms|
+--------+



In [ ]:
titanic_df = titanic_df.replace(['Mlle','Mme', 'Ms', 'Dr','Major','Dona','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
               ['Miss','Miss','Miss','Mr','Mr',  'Mrs','Mrs', 'Mrs',  'Other',  'Other','Other','Mr','Mr','Mr'])

In [ ]:
titanic_df.select('Initial').distinct().show()

+-------+
|Initial|
+-------+
|   Miss|
|  Other|
| Master|
|     Mr|
|    Mrs|
+-------+



In [ ]:
titanic_df.groupby('Initial').avg('Age').collect()

[Row(Initial='Miss', avg(Age)=21.83450186915888),
 Row(Initial='Other', avg(Age)=44.92307692307692),
 Row(Initial='Master', avg(Age)=5.482703773584906),
 Row(Initial='Mr', avg(Age)=32.545531197301855),
 Row(Initial='Mrs', avg(Age)=37.04624277456647)]

# **2. Transformations**

### Impute missing values in age feature based on average age of Initials

In [ ]:
titanic_df = titanic_df.withColumn('Age',when((titanic_df['Initial'] == 'Miss') & (titanic_df['Age'].isNull()), 22).otherwise(titanic_df['Age']))
titanic_df = titanic_df.withColumn('Age',when((titanic_df['Initial'] == 'Other') & (titanic_df['Age'].isNull()), 44).otherwise(titanic_df['Age']))
titanic_df = titanic_df.withColumn('Age',when((titanic_df['Initial'] == 'Master') & (titanic_df['Age'].isNull()), 5).otherwise(titanic_df['Age']))
titanic_df = titanic_df.withColumn('Age',when((titanic_df['Initial'] == 'Mr') & (titanic_df['Age'].isNull()), 33).otherwise(titanic_df['Age']))
titanic_df = titanic_df.withColumn('Age',when((titanic_df['Initial'] == 'Mrs') & (titanic_df['Age'].isNull()), 37).otherwise(titanic_df['Age']))


In [ ]:
titanic_df.filter(titanic_df.Age==5).select('Initial').show()

+-------+
|Initial|
+-------+
|   Miss|
| Master|
|   Miss|
|   Miss|
| Master|
|   Miss|
| Master|
| Master|
| Master|
| Master|
| Master|
| Master|
| Master|
+-------+



In [ ]:
titanic_df.show()

+------+--------------------+------+------+-----+-----+--------+--------+-------+--------+--------+-------+
|pclass|                name|   sex|   Age|sibsp|parch|  ticket|    fare|  cabin|embarked|survived|Initial|
+------+--------------------+------+------+-----+-----+--------+--------+-------+--------+--------+-------+
|     1|Allen, Miss. Elis...|female|  29.0|    0|    0|   24160|211.3375|     B5|       S|       1|   Miss|
|     1|Allison, Master. ...|  male|0.9167|    1|    2|  113781|  151.55|C22 C26|       S|       1| Master|
|     1|Allison, Miss. He...|female|   2.0|    1|    2|  113781|  151.55|C22 C26|       S|       0|   Miss|
|     1|Allison, Mr. Huds...|  male|  30.0|    1|    2|  113781|  151.55|C22 C26|       S|       0|     Mr|
|     1|Allison, Mrs. Hud...|female|  25.0|    1|    2|  113781|  151.55|C22 C26|       S|       0|    Mrs|
|     1| Anderson, Mr. Harry|  male|  48.0|    0|    0|   19952|   26.55|    E12|       S|       1|     Mr|
|     1|Andrews, Miss. Ko...

In [ ]:
titanic_df.groupby('Embarked').count().show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|  123|
|    NULL|    2|
|       C|  270|
|       S|  914|
+--------+-----+



In [ ]:
titanic_df = titanic_df.na.fill({'Embarked' : 'S'})

In [ ]:
titanic_df = titanic_df.drop('Cabin')

In [ ]:
titanic_df.show()

+------+--------------------+------+------+-----+-----+--------+--------+--------+--------+-------+
|pclass|                name|   sex|   Age|sibsp|parch|  ticket|    fare|embarked|survived|Initial|
+------+--------------------+------+------+-----+-----+--------+--------+--------+--------+-------+
|     1|Allen, Miss. Elis...|female|  29.0|    0|    0|   24160|211.3375|       S|       1|   Miss|
|     1|Allison, Master. ...|  male|0.9167|    1|    2|  113781|  151.55|       S|       1| Master|
|     1|Allison, Miss. He...|female|   2.0|    1|    2|  113781|  151.55|       S|       0|   Miss|
|     1|Allison, Mr. Huds...|  male|  30.0|    1|    2|  113781|  151.55|       S|       0|     Mr|
|     1|Allison, Mrs. Hud...|female|  25.0|    1|    2|  113781|  151.55|       S|       0|    Mrs|
|     1| Anderson, Mr. Harry|  male|  48.0|    0|    0|   19952|   26.55|       S|       1|     Mr|
|     1|Andrews, Miss. Ko...|female|  63.0|    1|    0|   13502| 77.9583|       S|       1|   Miss|


In [ ]:
titanic_df.printSchema()

root
 |-- pclass: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- sibsp: integer (nullable = true)
 |-- parch: integer (nullable = true)
 |-- ticket: string (nullable = true)
 |-- fare: double (nullable = true)
 |-- embarked: string (nullable = false)
 |-- survived: integer (nullable = true)
 |-- Initial: string (nullable = true)



In [ ]:
titanic_df = titanic_df.withColumn('Family_Size',col('SibSp')+col('Parch'))

In [ ]:
titanic_df.groupby('Family_Size').count().show()

+-----------+-----+
|Family_Size|count|
+-----------+-----+
|          1|  235|
|          6|   16|
|          3|   43|
|          5|   25|
|          4|   22|
|          7|    8|
|         10|   11|
|          2|  159|
|          0|  790|
+-----------+-----+



In [ ]:
titanic_df = titanic_df.withColumn('Alone',lit(0))
titanic_df = titanic_df.withColumn('Alone',when(titanic_df['Family_Size'] == 0,1).otherwise(titanic_df['Alone']))

In [ ]:
titanic_df.columns

['pclass',
 'name',
 'sex',
 'Age',
 'sibsp',
 'parch',
 'ticket',
 'fare',
 'embarked',
 'survived',
 'Initial',
 'Family_Size',
 'Alone']

In [ ]:
titanic_df.show()

+------+--------------------+------+------+-----+-----+--------+--------+--------+--------+-------+-----------+-----+
|pclass|                name|   sex|   Age|sibsp|parch|  ticket|    fare|embarked|survived|Initial|Family_Size|Alone|
+------+--------------------+------+------+-----+-----+--------+--------+--------+--------+-------+-----------+-----+
|     1|Allen, Miss. Elis...|female|  29.0|    0|    0|   24160|211.3375|       S|       1|   Miss|          0|    1|
|     1|Allison, Master. ...|  male|0.9167|    1|    2|  113781|  151.55|       S|       1| Master|          3|    0|
|     1|Allison, Miss. He...|female|   2.0|    1|    2|  113781|  151.55|       S|       0|   Miss|          3|    0|
|     1|Allison, Mr. Huds...|  male|  30.0|    1|    2|  113781|  151.55|       S|       0|     Mr|          3|    0|
|     1|Allison, Mrs. Hud...|female|  25.0|    1|    2|  113781|  151.55|       S|       0|    Mrs|          3|    0|
|     1| Anderson, Mr. Harry|  male|  48.0|    0|    0| 

In [ ]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(titanic_df) for column in ["sex","embarked","Initial"]]
pipeline = Pipeline(stages=indexers)
titanic_df = pipeline.fit(titanic_df).transform(titanic_df)

In [ ]:
titanic_df.show()

+------+--------------------+------+------+-----+-----+--------+--------+--------+--------+-------+-----------+-----+---------+--------------+-------------+
|pclass|                name|   sex|   Age|sibsp|parch|  ticket|    fare|embarked|survived|Initial|Family_Size|Alone|sex_index|embarked_index|Initial_index|
+------+--------------------+------+------+-----+-----+--------+--------+--------+--------+-------+-----------+-----+---------+--------------+-------------+
|     1|Allen, Miss. Elis...|female|  29.0|    0|    0|   24160|211.3375|       S|       1|   Miss|          0|    1|      1.0|           0.0|          1.0|
|     1|Allison, Master. ...|  male|0.9167|    1|    2|  113781|  151.55|       S|       1| Master|          3|    0|      0.0|           0.0|          3.0|
|     1|Allison, Miss. He...|female|   2.0|    1|    2|  113781|  151.55|       S|       0|   Miss|          3|    0|      1.0|           0.0|          1.0|
|     1|Allison, Mr. Huds...|  male|  30.0|    1|    2|  1

In [ ]:
titanic_df.printSchema()

root
 |-- pclass: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- sibsp: integer (nullable = true)
 |-- parch: integer (nullable = true)
 |-- ticket: string (nullable = true)
 |-- fare: double (nullable = true)
 |-- embarked: string (nullable = false)
 |-- survived: integer (nullable = true)
 |-- Initial: string (nullable = true)
 |-- Family_Size: integer (nullable = true)
 |-- Alone: integer (nullable = false)
 |-- sex_index: double (nullable = false)
 |-- embarked_index: double (nullable = false)
 |-- Initial_index: double (nullable = false)



In [ ]:
titanic_df = titanic_df.drop("PassengerId","Name","Ticket","Cabin","Embarked","Sex","Initial")

In [ ]:
titanic_df.show()

+------+------+-----+-----+--------+--------+-----------+-----+---------+--------------+-------------+
|pclass|   Age|sibsp|parch|    fare|survived|Family_Size|Alone|sex_index|embarked_index|Initial_index|
+------+------+-----+-----+--------+--------+-----------+-----+---------+--------------+-------------+
|     1|  29.0|    0|    0|211.3375|       1|          0|    1|      1.0|           0.0|          1.0|
|     1|0.9167|    1|    2|  151.55|       1|          3|    0|      0.0|           0.0|          3.0|
|     1|   2.0|    1|    2|  151.55|       0|          3|    0|      1.0|           0.0|          1.0|
|     1|  30.0|    1|    2|  151.55|       0|          3|    0|      0.0|           0.0|          0.0|
|     1|  25.0|    1|    2|  151.55|       0|          3|    0|      1.0|           0.0|          2.0|
|     1|  48.0|    0|    0|   26.55|       1|          0|    1|      0.0|           0.0|          0.0|
|     1|  63.0|    1|    0| 77.9583|       1|          1|    0|      1.0|

In [ ]:
null_columns_count_list = null_value_count(titanic_df)
null_columns_count_list
spark.createDataFrame(null_columns_count_list, ['Column_With_Null_Value', 'Null_Values_Count']).show()

+----------------------+-----------------+
|Column_With_Null_Value|Null_Values_Count|
+----------------------+-----------------+
|                  fare|                1|
+----------------------+-----------------+



In [ ]:
titanic_df.groupby('pclass').avg('fare').collect()

[Row(pclass=1, avg(fare)=87.50899164086687),
 Row(pclass=3, avg(fare)=13.302888700564957),
 Row(pclass=2, avg(fare)=21.1791963898917)]

In [ ]:
titanic_df = titanic_df.na.fill({'fare' : 13})

In [ ]:
feature = VectorAssembler(inputCols=titanic_df.columns[1:],outputCol='features')
feature_vector = feature.transform(titanic_df)

In [ ]:
feature_vector.show()

+------+------+-----+-----+--------+--------+-----------+-----+---------+--------------+-------------+--------------------+
|pclass|   Age|sibsp|parch|    fare|survived|Family_Size|Alone|sex_index|embarked_index|Initial_index|            features|
+------+------+-----+-----+--------+--------+-----------+-----+---------+--------------+-------------+--------------------+
|     1|  29.0|    0|    0|211.3375|       1|          0|    1|      1.0|           0.0|          1.0|[29.0,0.0,0.0,211...|
|     1|0.9167|    1|    2|  151.55|       1|          3|    0|      0.0|           0.0|          3.0|[0.9167,1.0,2.0,1...|
|     1|   2.0|    1|    2|  151.55|       0|          3|    0|      1.0|           0.0|          1.0|[2.0,1.0,2.0,151....|
|     1|  30.0|    1|    2|  151.55|       0|          3|    0|      0.0|           0.0|          0.0|(10,[0,1,2,3,5],[...|
|     1|  25.0|    1|    2|  151.55|       0|          3|    0|      1.0|           0.0|          2.0|[25.0,1.0,2.0,151...|
|     1|

In [ ]:
(traningData, testData) = feature_vector.randomSplit([0.8, 0.2], seed = 11)

# **3. Machine Learning in Spark using SparkML**

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol="survived", featuresCol="features")

lrModel = lr.fit(traningData)
lr_prediction = lrModel.transform(testData)
lr_prediction.select("prediction", "survived", "features").show()
evaluator = MulticlassClassificationEvaluator(labelCol="survived", predictionCol="prediction", metricName="accuracy")

+----------+--------+--------------------+
|prediction|survived|            features|
+----------+--------+--------------------+
|       1.0|       1|[13.0,2.0,2.0,262...|
|       1.0|       1|[14.0,1.0,2.0,120...|
|       1.0|       1|[17.0,0.0,2.0,110...|
|       1.0|       1|[17.0,1.0,0.0,57....|
|       1.0|       1|[17.0,1.0,0.0,108...|
|       1.0|       1|[18.0,1.0,0.0,53....|
|       1.0|       1|[18.0,1.0,0.0,60....|
|       1.0|       1|[19.0,0.0,0.0,30....|
|       1.0|       1|[21.0,0.0,0.0,77....|
|       0.0|       0|(10,[0,2,3,5],[21...|
|       0.0|       0|(10,[0,3,6,8],[22...|
|       1.0|       1|[22.0,0.0,2.0,49....|
|       0.0|       0|(10,[0,3,6],[23.0...|
|       1.0|       1|[24.0,0.0,0.0,69....|
|       0.0|       0|(10,[0,1,3,5],[24...|
|       1.0|       1|[27.0,1.0,1.0,247...|
|       0.0|       0|(10,[0,3,6],[28.0...|
|       0.0|       0|(10,[0,3,6],[29.0...|
|       0.0|       0|(10,[0,1,3,5],[29...|
|       1.0|       1|[30.0,1.0,0.0,57....|
+----------

In [ ]:
lr_accuracy = evaluator.evaluate(lr_prediction)
print('Accuracy of LogisticRegression is = %g'% (lr_accuracy))
print('Test Error of LogisticRegression = %g'% (1.0 - lr_accuracy))

Accuracy of LogisticRegression is = 1
Test Error of LogisticRegression = 0


In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(labelCol='survived', featuresCol='features')
dt_model = dt.fit(traningData)
dt_prediction = dt_model.transform(testData)
dt_prediction.select('prediction', 'survived', 'features').show()

+----------+--------+--------------------+
|prediction|survived|            features|
+----------+--------+--------------------+
|       1.0|       1|[13.0,2.0,2.0,262...|
|       1.0|       1|[14.0,1.0,2.0,120...|
|       1.0|       1|[17.0,0.0,2.0,110...|
|       1.0|       1|[17.0,1.0,0.0,57....|
|       1.0|       1|[17.0,1.0,0.0,108...|
|       1.0|       1|[18.0,1.0,0.0,53....|
|       1.0|       1|[18.0,1.0,0.0,60....|
|       1.0|       1|[19.0,0.0,0.0,30....|
|       1.0|       1|[21.0,0.0,0.0,77....|
|       0.0|       0|(10,[0,2,3,5],[21...|
|       0.0|       0|(10,[0,3,6,8],[22...|
|       1.0|       1|[22.0,0.0,2.0,49....|
|       0.0|       0|(10,[0,3,6],[23.0...|
|       1.0|       1|[24.0,0.0,0.0,69....|
|       0.0|       0|(10,[0,1,3,5],[24...|
|       1.0|       1|[27.0,1.0,1.0,247...|
|       0.0|       0|(10,[0,3,6],[28.0...|
|       0.0|       0|(10,[0,3,6],[29.0...|
|       0.0|       0|(10,[0,1,3,5],[29...|
|       1.0|       1|[30.0,1.0,0.0,57....|
+----------

In [ ]:
lr_accuracy = evaluator.evaluate(lr_prediction)
print("Accuracy of LogisticRegression is = %g"% (lr_accuracy))
print("Test Error of LogisticRegression = %g " % (1.0 - lr_accuracy))

Accuracy of LogisticRegression is = 1
Test Error of LogisticRegression = 0 


In [ ]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol='survived', featuresCol='features')
rf_model = rf.fit(traningData)
rf_prediction = rf_model.transform(testData)
rf_prediction.select('prediction', 'survived', 'features').show()

+----------+--------+--------------------+
|prediction|survived|            features|
+----------+--------+--------------------+
|       1.0|       1|[13.0,2.0,2.0,262...|
|       1.0|       1|[14.0,1.0,2.0,120...|
|       1.0|       1|[17.0,0.0,2.0,110...|
|       1.0|       1|[17.0,1.0,0.0,57....|
|       1.0|       1|[17.0,1.0,0.0,108...|
|       1.0|       1|[18.0,1.0,0.0,53....|
|       1.0|       1|[18.0,1.0,0.0,60....|
|       1.0|       1|[19.0,0.0,0.0,30....|
|       1.0|       1|[21.0,0.0,0.0,77....|
|       0.0|       0|(10,[0,2,3,5],[21...|
|       0.0|       0|(10,[0,3,6,8],[22...|
|       1.0|       1|[22.0,0.0,2.0,49....|
|       0.0|       0|(10,[0,3,6],[23.0...|
|       1.0|       1|[24.0,0.0,0.0,69....|
|       0.0|       0|(10,[0,1,3,5],[24...|
|       1.0|       1|[27.0,1.0,1.0,247...|
|       0.0|       0|(10,[0,3,6],[28.0...|
|       0.0|       0|(10,[0,3,6],[29.0...|
|       0.0|       0|(10,[0,1,3,5],[29...|
|       1.0|       1|[30.0,1.0,0.0,57....|
+----------

In [ ]:
rf_accuracy = evaluator.evaluate(rf_prediction)
print("Accuracy of RandomForestClassifier is = %g"% (rf_accuracy))
print("Test Error of RandomForestClassifier = %g " % (1.0 - rf_accuracy))

Accuracy of RandomForestClassifier is = 1
Test Error of RandomForestClassifier = 0 


In [ ]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(labelCol="survived", featuresCol="features",maxIter=10)
gbt_model = gbt.fit(traningData)
gbt_prediction = gbt_model.transform(testData)
gbt_prediction.select("prediction", "survived", "features").show()

+----------+--------+--------------------+
|prediction|survived|            features|
+----------+--------+--------------------+
|       1.0|       1|[13.0,2.0,2.0,262...|
|       1.0|       1|[14.0,1.0,2.0,120...|
|       1.0|       1|[17.0,0.0,2.0,110...|
|       1.0|       1|[17.0,1.0,0.0,57....|
|       1.0|       1|[17.0,1.0,0.0,108...|
|       1.0|       1|[18.0,1.0,0.0,53....|
|       1.0|       1|[18.0,1.0,0.0,60....|
|       1.0|       1|[19.0,0.0,0.0,30....|
|       1.0|       1|[21.0,0.0,0.0,77....|
|       0.0|       0|(10,[0,2,3,5],[21...|
|       0.0|       0|(10,[0,3,6,8],[22...|
|       1.0|       1|[22.0,0.0,2.0,49....|
|       0.0|       0|(10,[0,3,6],[23.0...|
|       1.0|       1|[24.0,0.0,0.0,69....|
|       0.0|       0|(10,[0,1,3,5],[24...|
|       1.0|       1|[27.0,1.0,1.0,247...|
|       0.0|       0|(10,[0,3,6],[28.0...|
|       0.0|       0|(10,[0,3,6],[29.0...|
|       0.0|       0|(10,[0,1,3,5],[29...|
|       1.0|       1|[30.0,1.0,0.0,57....|
+----------

In [ ]:
gbt_accuracy = evaluator.evaluate(gbt_prediction)
print("Accuracy of Gradient-boosted tree classifie is = %g"% (gbt_accuracy))
print("Test Error of Gradient-boosted tree classifie %g"% (1.0 - gbt_accuracy))

Accuracy of Gradient-boosted tree classifie is = 1
Test Error of Gradient-boosted tree classifie 0


In [ ]:
from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes(labelCol="survived", featuresCol="features")
nb_model = nb.fit(traningData)
nb_prediction = nb_model.transform(testData)
nb_prediction.select("prediction", "survived", "features").show()

+----------+--------+--------------------+
|prediction|survived|            features|
+----------+--------+--------------------+
|       1.0|       1|[13.0,2.0,2.0,262...|
|       1.0|       1|[14.0,1.0,2.0,120...|
|       1.0|       1|[17.0,0.0,2.0,110...|
|       1.0|       1|[17.0,1.0,0.0,57....|
|       1.0|       1|[17.0,1.0,0.0,108...|
|       1.0|       1|[18.0,1.0,0.0,53....|
|       1.0|       1|[18.0,1.0,0.0,60....|
|       1.0|       1|[19.0,0.0,0.0,30....|
|       1.0|       1|[21.0,0.0,0.0,77....|
|       1.0|       0|(10,[0,2,3,5],[21...|
|       1.0|       0|(10,[0,3,6,8],[22...|
|       1.0|       1|[22.0,0.0,2.0,49....|
|       1.0|       0|(10,[0,3,6],[23.0...|
|       1.0|       1|[24.0,0.0,0.0,69....|
|       1.0|       0|(10,[0,1,3,5],[24...|
|       1.0|       1|[27.0,1.0,1.0,247...|
|       1.0|       0|(10,[0,3,6],[28.0...|
|       0.0|       0|(10,[0,3,6],[29.0...|
|       1.0|       0|(10,[0,1,3,5],[29...|
|       1.0|       1|[30.0,1.0,0.0,57....|
+----------

In [ ]:
nb_accuracy = evaluator.evaluate(nb_prediction)
print("Accuracy of NaiveBayes is  = %g"% (nb_accuracy))
print("Test Error of NaiveBayes  = %g " % (1.0 - nb_accuracy))

Accuracy of NaiveBayes is  = 0.766667
Test Error of NaiveBayes  = 0.233333 


In [ ]:
from pyspark.ml.classification import LinearSVC
svm = LinearSVC(labelCol="survived", featuresCol="features")
svm_model = svm.fit(traningData)
svm_prediction = svm_model.transform(testData)
svm_prediction.select("prediction", "survived", "features").show()

+----------+--------+--------------------+
|prediction|survived|            features|
+----------+--------+--------------------+
|       1.0|       1|[13.0,2.0,2.0,262...|
|       1.0|       1|[14.0,1.0,2.0,120...|
|       1.0|       1|[17.0,0.0,2.0,110...|
|       1.0|       1|[17.0,1.0,0.0,57....|
|       1.0|       1|[17.0,1.0,0.0,108...|
|       1.0|       1|[18.0,1.0,0.0,53....|
|       1.0|       1|[18.0,1.0,0.0,60....|
|       1.0|       1|[19.0,0.0,0.0,30....|
|       1.0|       1|[21.0,0.0,0.0,77....|
|       0.0|       0|(10,[0,2,3,5],[21...|
|       0.0|       0|(10,[0,3,6,8],[22...|
|       1.0|       1|[22.0,0.0,2.0,49....|
|       0.0|       0|(10,[0,3,6],[23.0...|
|       1.0|       1|[24.0,0.0,0.0,69....|
|       0.0|       0|(10,[0,1,3,5],[24...|
|       1.0|       1|[27.0,1.0,1.0,247...|
|       0.0|       0|(10,[0,3,6],[28.0...|
|       0.0|       0|(10,[0,3,6],[29.0...|
|       0.0|       0|(10,[0,1,3,5],[29...|
|       1.0|       1|[30.0,1.0,0.0,57....|
+----------

In [ ]:
svm_accuracy = evaluator.evaluate(svm_prediction)
print("Accuracy of Support Vector Machine is = %g"% (svm_accuracy))
print("Test Error of Support Vector Machine = %g " % (1.0 - svm_accuracy))

Accuracy of Support Vector Machine is = 1
Test Error of Support Vector Machine = 0 
